In [36]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from datetime import datetime

In [37]:
api_key = 'QFZF2CEU5FAI6YIW'

sym = input("What is the ticker you are looking for? ")

What is the ticker you are looking for? NKE


In [38]:
ticker = sym
date = 'year1month1'
api_key = 'QFZF2CEU5FAI6YIW'

df_1 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

date = 'year1month2'
df_2 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

date = 'year1month3'
df_3 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

date = 'year1month4'
df_4 = pd.read_csv('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+ticker+'&interval=15min&slice='+date+'&apikey='+api_key+'&datatype=csv&outputsize=full')

In [39]:
df = pd.concat([df_1, df_2, df_3, df_4], ignore_index = True)
df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
df['Day_of_the_Week'] = df['time'].dt.day_name()

time_end = df['time'].iloc[0]
time_begin = df['time'].iloc[-1]

TB = str(time_begin)[0:10]
time_begin = TB +  str(' 6:30:00')

TE = str(time_end)[0:10]
time_end = TB +  str(' 13:00:00')

df.set_index('time')
df

,time,open,high,low,close,volume,Day_of_the_Week
0,2021-10-11 20:00:00,150.060000,150.060000,150.020000,150.030000,750,Monday
1,2021-10-11 19:45:00,150.030000,150.150000,150.020000,150.150000,1792,Monday
2,2021-10-11 19:30:00,150.070000,150.070000,150.060000,150.060000,295,Monday
3,2021-10-11 19:15:00,150.190000,150.190000,150.190000,150.190000,153,Monday
4,2021-10-11 17:15:00,150.150000,150.150000,150.150000,150.150000,228,Monday
...,...,...,...,...,...,...,...
3563,2021-06-14 07:00:00,132.023349,132.023349,132.023349,132.023349,201,Monday
3564,2021-06-14 06:45:00,132.043316,132.043316,132.043316,132.043316,4012,Monday
3565,2021-06-14 06:30:00,131.813693,131.813693,131.813693,131.813693,1703,Monday
3566,2021-06-14 05:00:00,131.683906,131.683906,131.683906,131.683906,236,Monday


In [40]:
# see how many days are within the the dataset. 

l = len(df['time'])
leng = int(l)-1

be = df['time'][0]
en = df['time'][leng]
delta = en - be
str_del = str(delta)
length = str_del[0:4]

length = abs(int(length))
print('There are' , length, 'days within the data set.')

There are 120 days within the data set.


In [41]:
beginning = pd.date_range(start = time_begin, end = TE, freq = 'D')
the_end = pd.date_range(start = time_end , end = TE, freq = 'D')

beginning = beginning.to_frame(index = False)
beginning.rename(columns={0 : 't'}, inplace = True)
the_end = the_end.to_frame(index = False)
the_end.rename(columns = {0 : 'ti'}, inplace = True)

first = beginning['t'].tolist()
second = the_end['ti'].tolist()

In [42]:
# define one day with 15 minute intervals beginning and ending on equities market.

# convert the one_day datetime index to a dataframe, then rename the column to match the master dataframe. 

# merging the one_day dataframe, with the master dataframe. Returns all information that has common dates, 
# due to inner join. 

# locating the individual record in the combo data frame based off of minimum value for low column. 
# returns index of minimum value for low column and stored in ind variable.

# new dataframe is created, and each individual record is located by integer, and concatenated to a new dataframe. 

### Function to Create Intraday DataFrame as CSV 

In [43]:
def go(beginning_time, end_time):
    total_time = len(beginning_time)
    i = 0
    while i < total_time:
        one_day = pd.date_range(start = beginning_time[i], end = end_time[i] , freq = '15min')
        single_day = one_day.to_frame(index = True)
        single_day.rename(columns={0 : 'time'}, inplace = True)
        combo = pd.merge(single_day, df, on = 'time', how = 'inner') 
        if not combo.empty:
            data = combo
            ind = data[['low']].idxmin()
            new = pd.DataFrame()
            record = data.iloc[ind]
            answer = pd.concat([new, record], ignore_index = True)
            answer.to_csv(sym+'_intraday.csv', index = False, mode = 'a')
        i += 1

In [44]:
go(first, second)